In [1]:
#! /usr/bin/env python

###################################################################################
# Copyright   2015, Pittsburgh Supercomputing Center (PSC).  All Rights Reserved. #
# =============================================================================== #
#                                                                                 #
# Permission to use, copy, and modify this software and its documentation without #
# fee for personal use within your organization is hereby granted, provided that  #
# the above copyright notice is preserved in all copies and that the copyright    #
# and this permission notice appear in supporting documentation.  All other       #
# restrictions and obligations are defined in the GNU Affero General Public       #
# License v3 (AGPL-3.0) located at http://www.gnu.org/licenses/agpl-3.0.html  A   #
# copy of the license is also provided in the top level of the source directory,  #
# in the file LICENSE.txt.                                                        #
#                                                                                 #
###################################################################################

import logging

import os.path
from phacsl.utils.collections.phacollections import SingletonMetaClass
import pyrheautils
from collections import deque, defaultdict
from facilitybase import CareTier
from tools_util import readModelInputs, getFacDict
import yaml

runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_ChicagoLand.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

totD = defaultdict(int)
nFacD = defaultdict(int)
for fac, rec in facDict.items():
    nFacD[rec['category']] += 1
    totD[rec['category']] += rec['meanPop']['value']
print 'totD: ', totD
print 'nFacD: ', nFacD

# Drop annoying community entries
dropL = []
for abbrev, rec in facDict.items():
    if rec['category'] == 'COMMUNITY':
        dropL.append(abbrev)
for abbrev in dropL:
    del facDict[abbrev]

print facDict.keys()



/home/welling/myEnv/local/lib/python2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


totD:  defaultdict(<type 'int'>, {'SNF': 44029.96048152038, 'HOSPITAL': 13839.558332146706, 'VSNF': 2359.043835616, 'COMMUNITY': 9082963, 'LTACH': 528.550274452})
nFacD:  defaultdict(<type 'int'>, {'SNF': 351, 'HOSPITAL': 90, 'VSNF': 12, 'COMMUNITY': 2260, 'LTACH': 9})
['HEAT_15600_S', 'BMO_10602_S', 'NORW_6016_S', 'BETH_5025_H', 'SOUT_2649_S', 'PRES_8001_S', 'CRES_14255_S', 'PERS_3900_S', 'COUN_2330_S', 'MID_4920_S', 'SOUT_1010_S', 'GLEN_4340_S', 'RESU_2380_S', 'ARBO_1512_S', 'GROS_6601_S', 'PRES_1100_S', 'PRES_1325_H', 'MANO_7850_S', 'FRAN_555_S', 'MICH_802_S', 'ST_3800_S', 'THE_110_S', 'MAYF_5905_S', 'GLEN_270_S', 'MANO_940_S', 'ALDE_2021_S', 'MAJO_7935_S', 'OAKT_1660_S', 'ALSH_2840_S', 'MANO_11860_S', 'EVAN_1300_S', 'ADVO_17800_H', 'HEND_1700_S', 'WEAL_150_S', 'IMPE_3300_S', 'MAJO_10352_S', 'OAKR_323_S', 'FOUN_1000_S', 'PRES_7435_H', 'GEOR_3623_S', 'CLAR_7433_S', 'MANO_600_S', 'MANO_9401_S', 'NORT_2650_H', 'PROV_13259_S', 'DOLT_14325_S', 'REGE_6631_S', 'GLEN_2451_V', 'GLEN_8333_V',

In [4]:
import phacsl.utils.formats.csv_tools as csv_tools
import phacsl.utils.formats.yaml_tools as yaml_tools
from pyrheautils import pathTranslate

trnsFacLOSFName = 'Matrices_LOS_09292016_cleaned_Facilities_LOS.csv'

trnsFacLOSPath = pathTranslate('$(MODELDIR)/%s' % trnsFacLOSFName)
with open(trnsFacLOSPath, 'rU') as f:
    tflKeys, tflRecs = csv_tools.parseCSV(f)
print tflKeys
tflCsvD = {r['UNIQUE_ID']: r for r in tflRecs}

[u'UNIQUE_ID', u'Distinct_Beneficiaries', u'N Obs', u'Sum', u'Mean', u'Minimum', u'Maximum', u'Median', u'Std Dev', u'Mode', u'Lower 95%', u'Upper 95%', u'25th Pctl', u'75th Pctl']


In [17]:
excludedFacPath = pathTranslate('$(MODELDIR)/facilityfacts_excluded')
keys, recs = yaml_tools.parse_all(excludedFacPath)
excludedFacD = {r['abbrev']: r for r in recs}
print excludedFacD.keys()

['WINC_6326_S', 'ANN_225_C', 'COOK_15900_H', 'SOME_5009_S', 'WESL_1751_S', 'JOHN_1367_S', 'SNOW_5000_S', 'COOK_500_H', 'KENW_7434_S', 'OAK_520_S', 'WEST_3240_H', 'WAUK_2615_H', 'THE_116_S', 'CAPI_7250_S', 'DEKA_1316_H', 'NEWT_300_S', 'MISE_6300_S', 'LAR_E_C', 'PHWD_3500_S']


In [25]:
with open(pathTranslate('$(MODELDIR)/direct_transfer_counts.yaml'), 'rU') as f:
    directTransferCts = yaml.load(f)
totTransIn = defaultdict(lambda: 0.0)
for src, rec in directTransferCts.items():
    for dst, ct in rec.items():
        totTransIn[dst] += ct
print totTransIn['LITT_80_S']
print directTransferCts['LITT_80_S']

5.0
OrderedDict([('ALEX_800_H', 1)])


In [32]:
catS = set()
facToCatD = {}
for src, rec in directTransferCts.items():
    facToCatD[src] = defaultdict(lambda: 0)
    for dst, ct in rec.items():
        if dst in facDict:
            cat = facDict[dst]['category']
            catS.add(cat)
            facToCatD[src][cat] += ct
        else:
            cat = excludedFacD[dst]['category']
            facToCatD[src][cat] += ct
print catS
print facToCatD['LITT_80_S']

set(['SNF', 'HOSPITAL', 'VSNF', 'LTACH'])
defaultdict(<function <lambda> at 0x7f5df235fde8>, {'HOSPITAL': 1})


In [36]:
from scipy.stats.distributions import rv_continuous, lognorm
import math
newFacDict = {}
directMtxFname = 'Matrices_LOS_09292016_cleaned.xlsx:Transfer3day'
transProv = ('%s translated by translate_direct_transfer_data.py d527d6a1 and marginalized by category'
             % directMtxFname)
transInProv = ('%s translated by translate_direct_transfer_data.py d527d6a1 summed over sources'
               % directMtxFname)
totDischProv = ('%s 8eb6e6c1 column "N Obs"' % trnsFacLOSFName)
for abbrev, rec in facDict.items():
    newR = rec.copy()
    if rec['category'] in ['NURSINGHOME', 'SNF', 'LTAC', 'LTACH', 'VSNF', 'HOSPITAL']:
        newR['totalDischarges'] = {'value': tflCsvD[abbrev]['N Obs'], 'prov': totDischProv}
        newR['totalTransfersIn'] = {'value': totTransIn[abbrev], 'prov': transInProv}
        newR['totalTransfersOut'] = [{'category': cat, 'count': {'value': ct, 'prov': transProv}}
                                     for cat, ct in facToCatD[abbrev].items()]
        #meanPopProv = "%s col 'AVERAGE DAILY CENSUS'" % snfChrFname
        #nBedsProv = "%s col 'TOTAL LICENSED BEDS'" % snfChrFname
        #csvR = snfCsvD[abbrev]
        #newR['meanPop'] = {'value': snfChrD[abbrev]['AVERAGE DAILY CENSUS'],
        #                  'prov': meanPopProv}
        #newR['nBeds'] = {'value': snfChrD[abbrev]['TOTAL LICENSED BEDS'],
        #                'prov': nBedsProv}
        #newR['meanLOSICU'] = {'value': 0.0, 'prov': 'NURSINGHOMEs do not have ICUs'}
        #newR['fracAdultPatientDaysICU'] = {'value': 0.0, 'prov': 'NURSINGHOMEs do not have ICUs'}
        ##losR = nhLOSD[abbrev]
        ##newR['losModel'] = {'negLogLikPerSample': -float(losR['lnLikPerSample'].strip('[]')),
        ##                    'nsamp': losR['nsamples'],
        ##                    'parms': [losR['k'], losR['mu'], losR['sigma'], losR['lmda']],
        ##                    'pdf': losR['pdf'].strip('"'),
        ##                    'prov': ('los_model_fit_fixedmean.py e308e809 applied to '
        ##                             'OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_09-29-2017_FINAL.xlsx')
        ##                    }
        #losR = wbNHLOSD[abbrev]
        #newR['losModel'] = {'parms': [1.0 - losR['k'], losR['shape2'], losR['scale2'],
        #                              losR['shape1'], losR['scale1']],
        #                    'pdf': '$0*weibull(k=$1, lmda=$2)+(1-$0)*weibull(k=$3, lmda=$4)',
        #                    'prov': 'nh-fitting-2weibull.R 14d707b7 based on OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_09-29-2017_FINAL_NH_LOS_Line_List.csv'}
        #snfChrR = snfChrD[abbrev]
        #newR['meanLOS'] = {'value': (365.*float(snfChrR['AVERAGE DAILY CENSUS'])
        #                             / float(snfChrR['"ANNUAL ADMISSIONS ADULT ONLY"'])),
        #                   'prov': ('%s 365*"AVERAGE DAILY CENSUS"/"ANNUAL ADMISSIONS ADULT ONLY"'
        #                            % snfChrFname)
        #                   }
    else:
        raise RuntimeError('unknown category %s for %s' % (rec['category'], abbrev))
        #transProv = ('(%s summed by dest category and normalized) * (%s total discharges * frac to NH or HOSP)'
        #             % (directMtxFname, hospDspFname))
        ##meanPopProv = "%s col 'AVERAGE DAILY CENSUS'" % hospChrFname
        ##nBedsProv = "%s col 'TOTAL LICENSED BEDS'" % hospChrFname
        #if abbrev in hospCsvD:
            #csvR = hospCsvD[abbrev]
            #totDsch = csvR['Total Annual Discharges']
            #totTrans = totDsch * (1.0 - (csvR[u'(4) Indirectly transferred to hospital within 365 days']
            #                             + csvR[u'(5) Discharged and NOT readmitted to hospital within 365 days']
            #                             + csvR[u'(1) Patient Died']))
            #tNHFrac = facToCatD[abbrev]['NURSINGHOME'] if 'NURSINGHOME' in facToCatD[abbrev] else 0.0
            #tLTACFrac = facToCatD[abbrev]['LTAC'] if 'LTAC' in facToCatD[abbrev] else 0.0
            #tHospFrac = facToCatD[abbrev]['HOSPITAL'] if 'HOSPITAL' in facToCatD[abbrev] else 0.0
            #newR['totalDischarges'] = {'value': totDsch,
            #                          'prov': '%s col B' % hospDspFname}
            #newR['totalTransfersOut'] = [
            #    {'category': 'NURSINGHOME', 'count': {'value': totTrans*tNHFrac, 'prov': transProv}},
            #    {'category': 'LTAC', 'count': {'value': totTrans*tLTACFrac, 'prov': transProv}},
            #    {'category': 'HOSPITAL', 'count': {'value': totTrans*tHospFrac, 'prov': transProv}}
            #]
            #newR['meanPop'] = {'value': hospChrD[abbrev][u'AVERAGE DAILY CENSUS ADULT ONLY'],
            #                'prov': meanPopProv}
            #newR['nBeds'] = {'value':hospChrD[abbrev]['TOTAL ADULT BEDS'],
            #                'prov': nBedsProv}
            #losMu = hospLOSD[abbrev]['LN_MEAN']
            #losSigma = hospLOSD[abbrev]['LN_STD_DEV']
            #losRV = lognorm(losSigma, scale=math.exp(losMu), loc=0.0)
            #meanLOS = losRV.mean()
            #newR['meanLOS'] = {'value': float(losRV.mean()), 
            #                   'prov':'%s mean of lognorm(sigma=LN_STD_DEV, mu=LN_MEAN)'% hospLOSFname}
            #newR['losModel'] = {'parms': [losMu, losSigma], 'pdf': 'lognorm(mu=$0,sigma=$1)',
            #                   'prov': '%s lognorm(sigma=LN_STD_DEV, mu=LN_MEAN)' % hospLOSFname}
            #newR['totalTransfersIn'] = {'value': totTransIn[abbrev],
            #                           'prov': ('translate_direct_transfer_data_OC.py d6d33794'
            #                                    + ' with input CSVs bdfa32e4')}
            #newR['scaleLengthOfStay'] = {'value': 1.0, 'prov': 'default is no scaling'}
    if 'totTransfersOut' in newR:
        del newR['totTransfersOut']

    newFacDict[abbrev] = newR
yaml_tools.save_all(pathTranslate('$(MODELDIR)/facilityfacts_updated'), newFacDict.values())

In [ ]:
hospDspFname = 'Hospital_and_NH_Disposition_Outcomes_for_RHEA_2.0_-_Adult_Only_-_UPDATED_-_11-15-2017_Hosp_Outcomes.csv'

hospPath = pathTranslate('$(MODELDIR)/%s' % hospDspFname)
with open(hospPath, 'rU') as f:
    hospKeys, hospRecs = csv_tools.parseCSV(f)
print hospKeys
hospCsvD = {r['HOSPITAL']: r for r in hospRecs}

In [ ]:
snfChrFname = 'OC_Hospital_and_Nursing_Home_Characteristics_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL_NH_CHAR.csv'

with open(pathTranslate('$(MODELDIR)/%s' % snfChrFname), 'rU') as f:
    snfChrKeys, snfChrRecs = csv_tools.parseCSV(f)
print snfChrKeys
snfChrD = {r['RHEA 2.0 CODE']: r for r in snfChrRecs}

In [ ]:
with open(pathTranslate('$(MODELDIR)/fac_to_cat_marginalized_direct.yaml'), 'rU') as f:
    facToCatD = yaml.load(f)
print facToCatD.keys()

In [ ]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_parms_fixedmean.csv'), 'rU') as f:
    nhLOSKeys, nhLOSRecs = csv_tools.parseCSV(f)
print nhLOSKeys
nhLOSD = {r['abbrev']: r for r in nhLOSRecs}
print nhLOSD['PSUP']

In [ ]:
hospChrFname = 'OC_Hospital_and_Nursing_Home_Characteristics_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL_HOSP_CHAR.csv'

with open(pathTranslate('$(MODELDIR)/%s' % hospChrFname), 'rU') as f:
    hospChrKeys, hospChrRecs = csv_tools.parseCSV(f)
print hospChrKeys
hospChrD = {r['RHEA 2.0 CODE']: r for r in hospChrRecs}

In [ ]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_nh_two_weibull.csv'), 'rU') as f:
    wbNHLOSKeys, wbNHLOSRecs = csv_tools.parseCSV(f)
print wbNHLOSKeys
wbNHLOSD = {r['abbrev']: r for r in wbNHLOSRecs}
print wbNHLOSD['PSUP']

In [ ]:
hospLOSFname = 'OC_Hospital_MRSA-Stratified_LOS_Distributions_for_RHEA_2.0_-_Adult_Only_-_09-18-2017_FINAL_MRSAMINUS_LOS_ALL.csv'
with open(pathTranslate('$(MODELDIR)/%s' % hospLOSFname), 'rU') as f:
    hospLOSKeys, hospLOSRecs = csv_tools.parseCSV(f)
print hospLOSKeys
hospLOSD = {r['CODE']: r for r in hospLOSRecs}


In [ ]:
from scipy.stats.distributions import rv_continuous, lognorm, weibull_min
import math
# 1.94 is the ln of the mean
# .83 is the ln of stdv
mu = 1.94
sigma = 0.83
rv = lognorm(sigma, math.exp(mu))
print rv.mean(), rv.median()


In [ ]:
mean = math.exp(1.94)
stdv = math.exp(0.83)
#mu = math.log(mean) - (0.5*stdv*stdv)
mu = mean - (0.5*stdv*stdv)
#rv = lognorm(stdv, scale=math.exp(mu), loc=0.0)
rv1 = lognorm(0.83, scale=mu, loc=0.0)
print rv1.median()

In [ ]:
mu = 1.94
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv2 = lognorm(sigma, scale=math.exp(mu), loc=0.0)
print 'mean:   ', rv2.mean()
print 'median: ', rv2.median()

In [ ]:
mu = 1.94
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv3 = lognorm(sigma, scale=math.exp(mu), loc=10.0)
print 'mean:   ', rv3.mean()
print 'median: ', rv3.median()

In [ ]:
k, shape1, scale1, shape2, scale2 = 0.5416364701, 0.6074742465, 165.0048283694, 1.3523950354, 20.4852828978

rv4 = weibull_min(shape1, scale=scale1)
rv5 = weibull_min(shape2, scale=scale2)
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv3 = lognorm(sigma, scale=math.exp(mu), loc=10.0)
print 'mean:   ', rv4.mean(), rv5.mean()
print 'median: ', rv4.median(), rv5.median()

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
axes = plt.subplot(111)
x = np.linspace(0, 31)
axes.plot(x, rv1.pdf(x), label='rv1')
axes.plot(x, rv2.pdf(x), label='rv2')
axes.plot(x, rv3.pdf(x), label='rv3')
axes.plot(x, rv5.pdf(x), label='rv5')
axes.legend()
plt.show()

In [ ]:
lnRV = lognorm(8.96443506e-01 , scale=math.exp(2.91480721e+00))
print lnRV.mean()

In [ ]:
from scipy.stats.distributions import expon
expRV = expon(scale=(1.0/2.36826336e-04))
print expRV.mean()